# Functions for retrieving data, making basic calculations, and running Monte Carlo simulations

## See [Basic usage](#Basic-usage) below if you just want to see how to use the module and don't care to read the code.

### Imports

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import datetime as dt
from pathlib import Path
import os
from datetime import datetime, timedelta

In [ ]:
from sqlalchemy import create_engine

In [ ]:
np.random.seed(42)

### Connect to DB

In [ ]:
# connectionString = "dbname='project01' user='pgadmin@fintech-postgres' host='fintech-postgres.postgres.database.azure.com' password='CgYRz!)[4pfB' port='5432' sslmode='true'"
engine = create_engine(
    "postgresql://pgadmin@fintech-postgres:CgYRz!)[4pfB@fintech-postgres.postgres.database.azure.com:5432/project01")

### Functions definitions

In [ ]:
def get_portfolio_daily_returns(portfolio_name, *, covid_phase):
    '''
    portfolio_name := string,
    covid_phase := boolean (whether to include data including dates 2/19/20 - 4/29/20)
    
    Query the postgres db.
    Run basic calculations.
    
    Return DF of portfolio daily returns that will be used in the rest of the analysis.
    '''
    
    q = f'''
        select portfolio_name, weight, "date" as close_date, 
            close_price, ticker_name, covidineffect 
        from portfolio p
            inner join portfolio_tickers pt
            on p.portfolio_id = pt.portfolio_id
            inner join price
            on pt.ticker_id = price.ticker_id
        where price.covidineffect = {covid_phase} and p.portfolio_name = '{portfolio_name}'
        '''
    portfolio_df = pd.read_sql(q, engine)
    
    # Pivot the table so dates are index
    portfolio_df = portfolio_df.pivot(index='close_date', columns='ticker_name', values='close_price')
    
    daily_returns = portfolio_df.pct_change()
    
    daily_returns = daily_returns.dropna()
    
    return daily_returns

In [ ]:
def portfolio_figures(pf_daily_returns):
    '''
    Returns tuple(last_close_price, returns_mean, returns_std)
    which can be passed in directly to `run_simulations`.
    '''
    return pf_daily_returns.iloc[-1], pf_daily_returns.mean(), pf_daily_returns.std()

In [ ]:
def run_simulations(pf_figures, num_records, num_simulations):
    '''
    pf_data := tuple(last_close_prices, returns_means, returns_stds)
    
    Runs `num_simulations` Monte Carlo simulations each of `num_records`.
    
    Returns DF of cumulative returns
    '''
    
    monte_carlo_cum_returns_df = pd.DataFrame()
    
    last_close_prices, returns_means, returns_stds = pf_figures
    
    for m in range(num_simulations):
        simulated_prices_all = [[price] for price in last_close_prices]
        
        for _ in range(num_records):
            
            for i, prices in enumerate(simulated_prices_all):
                sim_price = prices[-1] * (1 + np.random.normal(returns_means[i], returns_stds[i]))
                prices.append(sim_price)

        monte_carlo_price_df = pd.DataFrame(dict(zip(last_close_prices.index, simulated_prices_all)))

        sim_daily_returns = monte_carlo_price_df.pct_change()

        weighted_returns = sim_daily_returns.dot([0.25] * 4)

        monte_carlo_cum_returns_df[m] = (1 + weighted_returns.fillna(0)).cumprod()
    
    return monte_carlo_cum_returns_df

## Basic usage

The main functionality here is wrapped in 3 functions: `get_portfolio_daily_returns`, `portfolio_figures`, and `run_simulations`. You can run `portfolio_figures?` to view the doc string or just scroll up and look over the function definition.

### Examples

Suppose we want to compare the cumulative returns of the 5 portfolios using just the data before COVID.
Here's how we could get those data:

In [ ]:
# List of portfolios
portfolios = ['pharma', 'food', 'communication', 'hedge', 'financial']

# Dictionary for storing the results of the simulations
sim_results = dict.fromkeys(portfolios)

# Do the simulations and store results
for pf in portfolios:
    # covid=False here since we don't want data after 2/19/2020
    # Also notice the keyword `covid` is required here
    pf_returns = get_portfolio_daily_returns(pf, covid_phase=False)
    pf_figures = portfolio_figures(pf_returns)
    pf_sims = run_simulations(pf_figures, 30, 1) # 30 records, 1 simulation
    sim_results[pf] = pf_sims

In [ ]:
# Results of one of the portfolios
pharma_sim_results = sim_results['pharma']
pharma_sim_results.tail()

If we just want to get the (actual) daily returns of a portfolio, we can do that easily.

In [ ]:
financial_pf_real_daily_returns = get_portfolio_daily_returns('financial', covid_phase=False)
financial_pf_real_daily_returns.head()

Find the covariance of a stock:

In [ ]:
# BAC daily returns
bac = financial_pf_real_daily_returns['BAC']

# S&P 500 real returns
sp500_real_daily_returns = get_portfolio_daily_returns('S&P500', covid_phase=False)

# BAC covariance
bac.cov(sp500_real_daily_returns['SP500'])

## Tests

In [ ]:
# Tests here
